This example uses the default GLiNER model to perform named entity recognition.



In [8]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

text = """
Hi, a piano I have. Big and good, it is. Students use it, I want. How give it, can I? Olga
"""

labels = ["Person", "Money", "GiftType", "Interval", "Department"]

entities = model.predict_entities(text, labels, threshold=0.8)

for entity in entities:
    print(entity["text"], "=>", entity["label"])
    print(entity)

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Olga => Person
{'start': 87, 'end': 91, 'text': 'Olga', 'label': 'Person', 'score': 0.969987690448761}


We see that this model does a good job at extracting monitary, department, gift type and persons but fails when extracting intervals.